In [1]:
#importing the libraries required for the evaluation
import numpy as np
import rasterio
from pygeotools.lib import malib, warplib, iolib,geolib, timelib
import numpy.ma as ma
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
from imview.lib import pltlib
from osgeo import gdal, ogr

### Reading RED and SWIR Bands of Landsat ETM+

In [2]:
fn1='LE71480371999229AGS00_B3.TIF'
fn2='LE71480371999229AGS00_B5.TIF'
fn_list=[fn1,fn2]
ds_list = warplib.memwarp_multi_fn(fn_list, extent='intersection', res='max', t_srs=fn1)


Warping all inputs to the following:
Resolution: 30.0
Extent: [471585.0, 3566085.0, 706515.0, 3777615.0]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: cubic

1 of 2: LE71480371999229AGS00_B3.TIF
2 of 2: LE71480371999229AGS00_B5.TIF


In [5]:
band_ratio=ma.asarray(red,dtype=np.float32)/ma.asarray(swir,dtype=np.float32)

In [6]:
malib.print_stats(band_ratio)

count: 38002156 min: 0.00 max: 82.67 mean: 1.49 std: 2.28 med: 0.87 mad: 0.38 q1: 0.63 q2: 1.16 iqr: 0.53 mode: 1.00 p16: 0.57 p84: 1.36 spread: 0.39


(38002156,
 0.0045248871,
 82.666664,
 1.4908169478031559,
 2.2838692293065819,
 0.87000000476837158,
 0.37561373426914213,
 0.63380283117294312,
 1.1599999666213989,
 0.52619713544845581,
 1.0,
 0.57142859697341919,
 1.3582090139389038,
 0.39339020848274231)

### Limiting Band Ratio of Red/SWIR to 2.2 (Kaab et al. 2012) 

In [7]:
debris=ma.masked_outside(band_ratio,0, 2.2)

### Eliminating scattered Pixels as Noise using the mask_island function (malib, pygeotools)
## Be careful on the number of itereations, higher iterations might filter out valid continous data near edges as well

In [8]:
Debris_Islands_masked=malib.mask_islands(debris,iterations=2)

In [10]:
out_fn = 'Debris_cleaned_2.tif' 
iolib.writeGTiff(Debris_Islands_masked, out_fn, ds_list[1]) #Debris Cover Map with isolated pixels removed

### Reading shapefiles as updatable Features, Entering the percentage Debris covered per feature into a newly created field "ECW"

In [11]:
source = ogr.Open('Drung_drung.shp', update=True)
layer = source.GetLayer()
layer_defn = layer.GetLayerDefn()
new_field = ogr.FieldDefn("ECW", ogr.OFTReal)
layer.CreateField(new_field)
source = None

In [12]:
glac_shp_fn='Drung_drung.shp'
glac_shp_ds = ogr.Open(glac_shp_fn, update=True) #shapefile_dataset
glac_shp_lyr = glac_shp_ds.GetLayer() #layer of shapefile, all the operations are performed on it....
glac_shp_srs = glac_shp_lyr.GetSpatialRef() #Spatial_reference_of_shapefile
feat_count = glac_shp_lyr.GetFeatureCount() #Feature_Count
print("Input glacier polygon count: %i" % feat_count)


Input glacier polygon count: 27988


In [13]:
print str(glac_shp_srs)

PROJCS["WGS_1984_UTM_Zone_43N",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_84",6378137,298.257223563]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",75],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["Meter",1],
    AUTHORITY["EPSG","32643"]]


In [14]:
#dh_dt='WV_Carto_RateMaps-tile-0-last.tif' #input elevation_difference_map
#dh_dt='Filtered_dh_1_5_IQR.tif' #input elevation_difference_map
#z='B-tile-0.tif' #input DEM
#dh_dt_ds=gdal.Open(dh_dt) 
#z_ds=gdal.Open(z)
Landsat_geom = geolib.ds_geom_intersection(ds_list, t_srs=glac_shp_srs) #intersecting_extent
glac_shp_lyr.SetSpatialFilter(Landsat_geom) #spatial filtering of shapefiles within extent
feat_count = glac_shp_lyr.GetFeatureCount() #now only the polygons within the extents are considered
print("Filtered glacier polygon count: %i" % feat_count)
glac_shp_lyr.ResetReading()

Filtered glacier polygon count: 3036


In [15]:
debris_fieldname = "Percentage_Final"
glacnum_fieldname = "RGIId"

In [16]:
for n, feat in enumerate(glac_shp_lyr):
    print'Hi' #Check to see program flow...
    #Extracting glacier id/name from input shapefiles
    glacnum = feat.GetField(glacnum_fieldname)
    glacnum = float(glacnum.split('-')[-1])
    glacnum_fmt = '%0.5f'
    feat_fn = str(glacnum)
    print("\n%i of %i: %s\n" % (n+1, feat_count, feat_fn))
    glac_geom = feat.GetGeometryRef()
    glac_geom.AssignSpatialReference(glac_shp_srs)
    glac_geom_extent = geolib.geom_extent(glac_geom)
    glac_area = glac_geom.GetArea()
    print str(glac_area)
    #warping the Landsat_Band and Debris_islanded into common extents
    ds_list1 = warplib.memwarp_multi_fn([fn1, out_fn], res='max',r='bilinear',
            extent=glac_geom_extent, t_srs='EPSG:32643', verbose=False)
    glac_geom_mask = geolib.geom2mask(glac_geom, ds_list1[0])
    total_pixel = ma.array(iolib.ds_getma(ds_list1[0]), mask=glac_geom_mask)
    debris_pixel = ma.array(iolib.ds_getma(ds_list1[1]), mask=glac_geom_mask) 
    ds_res = geolib.get_res(ds_list[0])
    valid_Total_area = total_pixel.count()*ds_res[0]*ds_res[1]
    valid_Debris_area = debris_pixel.count()*ds_res[0]*ds_res[1]
    Debris_area_perc = (valid_Debris_area/valid_Total_area)*100
    print("Total_Area ="+str(valid_Total_area))
    print("Total_Debris_Area="+str(valid_Debris_area))
    print("Debris_Area_Percentage="+str(Debris_area_perc))
    glac_shp_lyr.SetFeature(feat)
    feat.SetField("ECW", Debris_area_perc)
    glac_shp_lyr.SetFeature(feat)
    #feat.SetField(debris_fieldname, Debris_area_perc)
    #layer.SetFeature(feat)
glac_shp_ds=None

Hi

1 of 3036: 14.1301

168333.104884

Warping all inputs to the following:
Resolution: 30.0
Extent: [691529.9500321876, 3643508.249871153, 692440.1184931222, 3643862.7922288757]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 12 ns: 30 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 12 ns: 30 res: 30.000
Total_Area =191700.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

2 of 3036: 14.13011

194819.574487

Warping all inputs to the following:
Resolution: 30.0
Extent: [691080.078614614, 3644326.7232036507, 691630.4918816497, 3645005.794552535]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 23 ns: 18 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 23 ns: 18 res: 30.000
Total_Area =213300.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3 of 3036: 14.13062

5291806.80113

Warping all inputs to the following:
Resolut

2 of 2: Debris_cleaned_2.tif
nl: 32 ns: 23 res: 30.000
Total_Area =355500.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

24 of 3036: 14.13098

114429.548689

Warping all inputs to the following:
Resolution: 30.0
Extent: [696134.2553361219, 3618534.5824795538, 696674.0064067575, 3618820.3330347417]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 10 ns: 18 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 10 ns: 18 res: 30.000
Total_Area =125100.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

25 of 3036: 14.13099

1102726.26737

Warping all inputs to the following:
Resolution: 30.0
Extent: [696642.2562847527, 3618979.661719368, 698552.5518275453, 3620164.419103155]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 39 ns: 64 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 39 ns: 64 res: 30.000
Total_Area =1171800.0
Tot

2 of 2: Debris_cleaned_2.tif
nl: 9 ns: 14 res: 30.000
Total_Area =88200.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

48 of 3036: 14.13134

459330.145721

Warping all inputs to the following:
Resolution: 30.0
Extent: [685802.3916563799, 3615583.084645087, 687627.983469257, 3616886.380574734]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 43 ns: 61 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 43 ns: 61 res: 30.000
Total_Area =523800.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

49 of 3036: 14.13135

1668212.10709

Warping all inputs to the following:
Resolution: 30.0
Extent: [683833.0956694648, 3614947.6238170792, 686108.3216763625, 3616884.343051361]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 65 ns: 76 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 65 ns: 76 res: 30.000
Total_Area =1814400.0
Total_D

2 of 2: Debris_cleaned_2.tif
nl: 30 ns: 11 res: 30.000
Total_Area =224100.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

67 of 3036: 14.14387

151692.858608

Warping all inputs to the following:
Resolution: 30.0
Extent: [691234.8403182761, 3765004.5766071607, 691558.6171890564, 3765681.9345183554]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 23 ns: 11 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 23 ns: 11 res: 30.000
Total_Area =171000.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

68 of 3036: 14.14388

63809.2215547

Warping all inputs to the following:
Resolution: 30.0
Extent: [690512.7256473354, 3765105.883882108, 690752.883527915, 3765541.1221292904]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 15 ns: 8 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 15 ns: 8 res: 30.000
Total_Area =73800.0
Total_D

Total_Area =273600.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

88 of 3036: 14.14611

419508.748256

Warping all inputs to the following:
Resolution: 30.0
Extent: [705971.1958313582, 3723962.1200152365, 706840.1060616014, 3725461.2300607585]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 50 ns: 29 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 50 ns: 29 res: 30.000
Total_Area =195300.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

89 of 3036: 14.14612

87246.9241011

Warping all inputs to the following:
Resolution: 30.0
Extent: [704635.7651130883, 3723482.635892079, 704980.321482677, 3723890.042958144]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 14 ns: 11 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 14 ns: 11 res: 30.000
Total_Area =94500.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

90 of 30

...

Hi

2939 of 3036: 14.18948

67769683.9921

Warping all inputs to the following:
Resolution: 30.0
Extent: [615787.7770008611, 3725284.0870822677, 627502.0981952515, 3746042.891105241]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 692 ns: 390 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 692 ns: 390 res: 30.000
Total_Area =71881200.0
Total_Debris_Area=7136100.0
Debris_Area_Percentage=9.92763059047
Hi

2940 of 3036: 14.18949

45532339.3474

Warping all inputs to the following:
Resolution: 30.0
Extent: [617730.9267628784, 3718865.2492141137, 631394.6167863439, 3730329.6944040465]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 382 ns: 455 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 382 ns: 455 res: 30.000
Total_Area =49093200.0
Total_Debris_Area=17072100.0
Debris_Area_Percentage=34.7748771724
Hi

2941 of 3036: 14.1895

534

nl: 223 ns: 201 res: 30.000
Total_Area =8823600.0
Total_Debris_Area=3517200.0
Debris_Area_Percentage=39.8612811098
Hi

2958 of 3036: 14.18967

279281.983142

Warping all inputs to the following:
Resolution: 30.0
Extent: [602195.4193711206, 3718935.800142047, 603392.6075702485, 3719335.55869674]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 13 ns: 40 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 13 ns: 40 res: 30.000
Total_Area =309600.0
Total_Debris_Area=18000.0
Debris_Area_Percentage=5.81395348837
Hi

2959 of 3036: 14.18968

186878.688361

Warping all inputs to the following:
Resolution: 30.0
Extent: [603784.0094587886, 3722397.6729597664, 604271.0157556604, 3723048.2060013185]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 22 ns: 16 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 22 ns: 16 res: 30.000
Total_Area =202500

Total_Area =259200.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

2980 of 3036: 14.19546

339693.373916

Warping all inputs to the following:
Resolution: 30.0
Extent: [548657.4470079287, 3776138.955671426, 549522.0607787014, 3776728.615723473]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 20 ns: 29 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 20 ns: 29 res: 30.000
Total_Area =363600.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

2981 of 3036: 14.19547

170579.183762

Warping all inputs to the following:
Resolution: 30.0
Extent: [548533.7730168385, 3773646.703786568, 549110.182168988, 3774302.6175873135]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 22 ns: 19 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 22 ns: 19 res: 30.000
Total_Area =191700.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

2982

Total_Area =344700.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3000 of 3036: 14.19723

105519.36599

Warping all inputs to the following:
Resolution: 30.0
Extent: [556483.8170058687, 3773712.8605051427, 556880.4795303623, 3774203.6463740785]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 16 ns: 13 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 16 ns: 13 res: 30.000
Total_Area =117000.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3001 of 3036: 14.20023

96691.2478475

Warping all inputs to the following:
Resolution: 30.0
Extent: [529743.6777294003, 3777400.4358509136, 530162.1903166146, 3778071.064376584]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 22 ns: 14 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 22 ns: 14 res: 30.000
Total_Area =23400.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

300

Total_Area =111600.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3023 of 3036: 14.20349

61606.215836

Warping all inputs to the following:
Resolution: 30.0
Extent: [484460.29573624814, 3703032.46308418, 484754.9935998984, 3703291.4399909913]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 9 ns: 10 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 9 ns: 10 res: 30.000
Total_Area =69300.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3024 of 3036: 14.2035

79350.3998078

Warping all inputs to the following:
Resolution: 30.0
Extent: [478646.71082496893, 3705738.325133388, 478977.12970961095, 3706086.604475579]
Projection: '+proj=utm +zone=43 +datum=WGS84 +units=m +no_defs '
Resampling alg: bilinear

1 of 2: LE71480371999229AGS00_B3.TIF
nl: 12 ns: 11 res: 30.000
2 of 2: Debris_cleaned_2.tif
nl: 12 ns: 11 res: 30.000
Total_Area =86400.0
Total_Debris_Area=0.0
Debris_Area_Percentage=0.0
Hi

3025 of